In [54]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasRegressor

In [55]:
# Load the dataset
data = pd.read_csv('C:/Users/user/Desktop/MSc Folder/Masters Research/Modelling Data/Modelling data.csv')

# Inspect the first few rows
print(data.head())

   Fly ash  Sodium Silicate  Sodium Hydroxide  Molarity  Coarse Aggregate  \
0    408.0            103.0              41.0         8            1294.0   
1    408.0            103.0              41.0         8            1294.0   
2    408.0            103.0              41.0         8            1294.0   
3    408.0            103.0              41.0         8            1294.0   
4    408.0            103.0              41.0        14            1294.0   

   Fine Aggregate  Curing Days  Curing Temperature  SS/SH Ratio  Extra Water  \
0           554.0          7.0                  30          2.0          0.0   
1           554.0          7.0                  45          2.0          0.0   
2           554.0          7.0                  75          2.0          0.0   
3           554.0          7.0                  90          2.0          0.0   
4           554.0          7.0                  30          2.0         22.5   

   Compressive Strength  
0                  43.0  
1   

In [56]:
# Define the features (X) and target (y)
X = data.drop(columns=['Compressive Strength'])  # All independent variables
y = data['Compressive Strength']  # Dependent variable
# Check the processed data
print(X.head())

   Fly ash  Sodium Silicate  Sodium Hydroxide  Molarity  Coarse Aggregate  \
0    408.0            103.0              41.0         8            1294.0   
1    408.0            103.0              41.0         8            1294.0   
2    408.0            103.0              41.0         8            1294.0   
3    408.0            103.0              41.0         8            1294.0   
4    408.0            103.0              41.0        14            1294.0   

   Fine Aggregate  Curing Days  Curing Temperature  SS/SH Ratio  Extra Water  
0           554.0          7.0                  30          2.0          0.0  
1           554.0          7.0                  45          2.0          0.0  
2           554.0          7.0                  75          2.0          0.0  
3           554.0          7.0                  90          2.0          0.0  
4           554.0          7.0                  30          2.0         22.5  


In [57]:
#Spliting Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# Function to create the model
def create_model(optimizer='adam', init='uniform'):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))  # Input layer
    model.add(Dense(32, kernel_initializer=init, activation='relu'))                             # Hidden layer
    model.add(Dense(16, kernel_initializer=init, activation='relu'))                             # Hidden layer
    model.add(Dense(1, kernel_initializer=init))                                                 # Output layer
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

# Create KerasRegressor using scikeras
model = KerasRegressor(model=create_model, verbose=0)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'batch_size': [10, 20, 50],              # Batch sizes for training
    'epochs': [50, 100, 200],                # Number of epochs
    'optimizer': ['adam', 'rmsprop'],        # Optimizers
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

# Train the model with GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


GridSearchCV(cv=5,
             estimator=KerasRegressor(model=<function create_model at 0x00000281BEC56FC0>, verbose=0),
             n_jobs=-1,
             param_grid={'batch_size': [10, 20, 50], 'epochs': [50, 100, 200],
                         'optimizer': ['adam', 'rmsprop']},
             scoring='neg_mean_squared_error', verbose=1)

In [59]:
# Evaluate the best model on the test data
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test accuracy: {:.2f}".format(test_accuracy))

Test accuracy: 0.48


In [60]:
# Predict on the test data
y_pred = best_model.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate R-squared (R²)
r2 = r2_score(y_test, y_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

# Print the results
print(f"MAPE: {mape:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

MAPE: 0.3813
MSE: 111.9899
R²: 0.4843
RMSE: 10.5825
